# Prediction

Machinery to predict targets for new cpds.
___

In [1]:
import pickle
import pandas as pd
import numpy as np
import scipy
import itertools
from support_functions import log_progress
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from cmapPy.pandasGEXpress import parse
from sklearn.preprocessing import StandardScaler

___
## Identify relevant signatures

In [30]:
# parameters
cpd_ids = ['BRD-K19227686', 
          'BRD-K65503129', 
          'BRD-K29905972', 
          'BRD-K59556282', 
          'BRD-K67298865',
          'BRD-K78930611',
         ]
          
cell_lines = ['A375', 'A549', 'MCF7', 'PC3']

In [37]:
# load all signature info
all_sig_info = pd.DataFrame.from_csv('checkpoint_files/all_sig_info.csv')

# extract the cpd of interest
idx = all_sig_info.pert_id.isin(cpd_ids)
cpd_sigs = all_sig_info[idx]

/Users/Nico/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (4,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [38]:
# refine to our cell lines of interest
cpd_sigs = cpd_sigs[cpd_sigs.cell_id.isin(cell_lines)].sort_values(['cell_id'])
cpd_sigs = cpd_sigs.drop(['distil_id', 'pert_dose', 'pert_dose_unit', 'pert_time', 'pert_time_unit'], axis=1)

In [41]:
# select representative signatures
repr_cpd_sigs = pd.DataFrame(columns=cpd_sigs.columns)
for pert_id in log_progress(cpd_ids):    
    for cell_id in cell_lines:
        candidate_sigs = cpd_sigs[(cpd_sigs.cell_id == cell_id) & (cpd_sigs.pert_id == pert_id)]
        repr_sig = candidate_sigs.loc[candidate_sigs['tas'].idxmax()]
        repr_cpd_sigs = repr_cpd_sigs.append(repr_sig)

## Extract query signatures

In [48]:
# extract profiles from GCTX
phase_1_sig_ids = parse('data/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx', col_meta_only=True).index.values
phase_2_sig_ids = parse('data/GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx', col_meta_only=True).index.values

# The GCTX files encode the sig IDS as strings of byte strings (why????) so we have to deal with that
cpd_sig_ids = [ str(s.encode('UTF8')) for s  in repr_cpd_sigs.sig_id.values ]
phase_1_cpd_sig_ids = [ s for s in cpd_sig_ids if s in phase_1_sig_ids ]
phase_2_cpd_sig_ids = [ s for s in cpd_sig_ids if s in phase_2_sig_ids ]

In [49]:
# extract the actual signatures
if phase_1_cpd_sig_ids:
    phase_1_cpd_sigs = parse('data/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx', cid=phase_1_cpd_sig_ids).data_df    
if phase_2_cpd_sig_ids:
    phase_2_cpd_sigs = parse('data/GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx', cid=phase_2_cpd_sig_ids).data_df
    phase_2_cpd_sigs.rename(lambda x: x[2:-1], inplace=True)

if (phase_1_cpd_sig_ids and phase_2_cpd_sig_ids):
    cpd_profiles = pd.concat([phase_1_top_4_cpd_sigs, phase_2_top_4_cpd_sigs], axis=1)
elif phase_1_cpd_sig_ids: 
    cpd_profiles = phase_1_cpd_sigs
elif phase_2_cpd_sig_ids:
    cpd_profiles = phase_2_cpd_sigs

In [52]:
# primary gene symbols and IDs
gene_info_1 = pd.read_csv('data/GSE92742_Broad_LINCS_gene_info.txt', sep='\t', header=0)
lm_genes = gene_info_1[gene_info_1['pr_is_lm'].astype(bool)]
lm_gene_ids = lm_genes['pr_gene_id'].astype(str).values

# discard all but landmark gene values
cpd_lm_profiles = cpd_profiles[cpd_profiles.index.isin(lm_gene_ids)].sort_index().copy()
cpd_lm_profiles.head()

# reformat the sig_ids so they are nice strings
cpd_lm_profiles = cpd_lm_profiles.T.rename(lambda x: x.lstrip('b\'').rstrip('\'')).T

___
## Construct features

In [59]:
# the known cpd-target interactions. shape: [244, 2]
top_4_known_interactions = pd.DataFrame.from_csv('checkpoint_files/top_4_known_interactions.csv')
# KD signature ID metadata. shape: [11948, 15]
repr_top_4_gold_kd_sigs = pd.DataFrame.from_csv('checkpoint_files/repr_top_4_gold_kd_sigs.csv')
# the actual KD signatures. shape: [978, 11948]
top_4_kd_lm_sigs = pd.DataFrame.from_csv('checkpoint_files/top_4_kd_lm_sigs.csv')

In [101]:
# create cpd-kd mapping
all_top_4_kds = repr_top_4_gold_kd_sigs.pert_iname.unique()
all_top_4_cpds = cpd_ids

# for now it doesn't really matter what label
cpd_ = []
kd_ = []

for kd in log_progress(all_top_4_kds):
    for cpd in all_top_4_cpds:
        cpd_.append(cpd)
        kd_.append(kd)
        
# store pairs in dataframe
cpd_kd_pairs_df = pd.DataFrame({'cpd': cpd_, 'kd': kd_})

### Direct correlation

In [102]:
# initialize empty dataframe to hold direct correlation values
dir_corr_df = pd.DataFrame(index=cpd_kd_pairs_df.index, columns=cell_lines)

# loop through cpd-target pairs, calculate correlations in each cell line
for index, row in log_progress(cpd_kd_pairs_df.iterrows(), every=100):
    cpd = row.cpd
    kd = row.kd
    
    for cell_line in cell_lines:
        cpd_sig_info = repr_cpd_sigs.query('pert_id == "{}" & cell_id == "{}"'.format(cpd,cell_line))
        kd_sig_info = repr_top_4_gold_kd_sigs.query('pert_iname == "{}" & cell_id == "{}"'.format(kd,cell_line))
        
        # extract signatures
        cpd_sig_id = cpd_sig_info.sig_id
        kd_sig_id = kd_sig_info.sig_id
        
        cpd_lm_sig = cpd_lm_profiles[cpd_sig_id].values
        kd_lm_sig = top_4_kd_lm_sigs[kd_sig_id].values
        
        #compute and store correlation
        corr = scipy.stats.pearsonr(cpd_lm_sig, kd_lm_sig)[0][0]
        dir_corr_df.at[index, cell_line] = corr

### Indirect correlation

In [98]:
# 710638 high confidence interactions
string_gene_interactions_700 = pd.DataFrame.from_csv('checkpoint_files/string_gene_interactions_700.csv')

In [95]:
# list of all KDs in top-4 cells, for reference
all_kds = repr_top_4_gold_kd_sigs.pert_iname.unique()

In [105]:
# initialize empty dataframe to hold direct correlation values
indir_max_corr_df = pd.DataFrame(index=cpd_kd_pairs_df.index, columns=cell_lines)
indir_min_corr_df = pd.DataFrame(index=cpd_kd_pairs_df.index, columns=cell_lines)
indir_avg_corr_df = pd.DataFrame(index=cpd_kd_pairs_df.index, columns=cell_lines)

# loop through cpd-target pairs, calculate correlations in each cell line
for index, row in log_progress(cpd_kd_pairs_df.iterrows(), every=10):
    cpd = row.cpd
    kd = row.kd
    
    # find the target's interaction partners that have KDs
    interaction_partners = string_gene_interactions_700.query('gene_1 == "{}"'.format(kd)).gene_2
    partner_kds = np.intersect1d(interaction_partners, all_kds)
    
    # compute corr with each partner in each cell line
    for cell_line in cell_lines:
        
        # extract the cpd signature
        cpd_sig_info = repr_cpd_sigs.query('pert_id == "{}" & cell_id == "{}"'.format(cpd,cell_line))
        cpd_sig_id = cpd_sig_info.sig_id
        cpd_lm_sig = cpd_lm_profiles[cpd_sig_id].values
       
        # initialize empty Series to hold corrs for all partners in this cell line
        pkd_corrs = pd.Series(index=partner_kds)
        
        for pkd in partner_kds:
            # extract the partner kd signature
            pkd_sig_info = repr_top_4_gold_kd_sigs.query('pert_iname == "{}" & cell_id == "{}"'.format(pkd,cell_line))
            pkd_sig_id = pkd_sig_info.sig_id
            pkd_lm_sig = top_4_kd_lm_sigs[pkd_sig_id].values

            #compute and store correlation
            corr = scipy.stats.pearsonr(cpd_lm_sig, pkd_lm_sig)[0][0]
            pkd_corrs[pkd] = corr
        
        # compute max, min, and average of the partner kd corrs
        max_pkd_corr = pkd_corrs.max()
        min_pkd_corr = pkd_corrs.min()
        avg_pkd_corr = pkd_corrs.mean()
        
        # store these in the appropriate data frames
        indir_max_corr_df.at[index, cell_line] = max_pkd_corr
        indir_min_corr_df.at[index, cell_line] = min_pkd_corr
        indir_avg_corr_df.at[index, cell_line] = avg_pkd_corr

### Remove missing malues

In [106]:
features_df = pd.concat([dir_corr_df, indir_max_corr_df, indir_min_corr_df, indir_avg_corr_df], axis=1)

In [107]:
# identify rows that have NaN entries
null_row_indeces = features_df.isnull().any(axis=1)
null_rows = features_df[null_row_indeces]
null_pairs = cpd_kd_pairs_df[null_row_indeces]

In [108]:
# remove potential targets with missing data
complete_row_indeces = ~null_row_indeces
complete_data_rows = features_df[complete_row_indeces]
complete_pair_rows = cpd_kd_pairs_df[complete_row_indeces]

### Scale the data

In [110]:
X = complete_data_rows
sc = StandardScaler()
sc.fit(X)
X_std = pd.DataFrame(sc.transform(X))

___
## Predict targets

In [112]:
# load the model
RF = pickle.load(open('model/RF_4.sav', 'rb'))

# run the classifier
results = complete_pair_rows.copy()
results['prob'] = RF.predict_proba(X_std)[:,1]
results.head()

,cpd,kd,prob
0,BRD-K19227686,EIF2AK3,0.016452
1,BRD-K65503129,EIF2AK3,0.000185
2,BRD-K29905972,EIF2AK3,0.010017
3,BRD-K59556282,EIF2AK3,0.000203
4,BRD-K67298865,EIF2AK3,0.000060


In [116]:
# sort potential targets by probability
for cpd in cpd_ids:
    cpd_results = results[results.cpd == cpd].copy()
    cpd_results = cpd_results.sort_values('prob', ascending=False).reset_index(drop=True)
    percentile = 1.0 - cpd_results[cpd_results.kd == 'STUB1'].index.values[0] / len(cpd_results)
    print(cpd, percentile)

BRD-K19227686 0.279365079365
BRD-K65503129 0.345238095238
BRD-K29905972 0.879365079365
BRD-K59556282 0.470238095238
BRD-K67298865 0.855555555556
BRD-K78930611 0.733333333333


In [21]:
percentile = 1.0 - results[results.target == 'STUB1'].index.values[0] / len(results)
percentile

0.47817460317460314

___
## Result rankings

<br>**CHIP cpds**
- BRD_K19227686 (phenolphthalein)  - STUB1: 0.19
- BRD_K65503129 (HSP90_inhibitor)  - STUB1: 0.48
- BRD_K29905972 (axitinib)         - STUB1: 
- BRD_K59556282 (BRD_K59556282)    - STUB1: 
- BRD_K67298865 (SB_431542)        - STUB1: 
- BRD_K78930611 (MW_STK33_2B)      - STUB1: 
